# Anytime heuristic search 

In [1]:
import copy
import math
from random import shuffle, randint
from heapq import heappop, heappush, heapify
import time

EPS = 0.00001

## Grid Search Domain Representation

In [2]:
class Map:
    '''
    Square grid map class represents the environment for our moving agent
        - width -- the number of columns in the grid
        - height -- the number of rows in the grid
        - cells -- the binary matrix, that represents the grid. 0 - cell is traversable, 1 - cell is blocked
    '''

    def __init__(self):
        '''
        Default constructor
        '''

        self.width = 0
        self.height = 0
        self.cells = []
    

    def ReadFromString(self, cellStr, width, height):
        '''
        Converting a string (with '#' representing obstacles and '.' representing free cells) to a grid
        '''
        self.width = width
        self.height = height
        self.cells = [[0 for _ in range(width)] for _ in range(height)]
        cellLines = cellStr.split("\n")
        i = 0
        j = 0
        for l in cellLines:
            if len(l) != 0:
                j = 0
                for c in l:
                    if c == '.':
                        self.cells[i][j] = 0
                    elif c == '#':
                        self.cells[i][j] = 1
                    else:
                        continue
                    j += 1
                if j != width:
                    raise Exception("Size Error. Map width = ", j, ", but must be", width )
                i += 1

        if i != height:
            raise Exception("Size Error. Map height = ", i, ", but must be", height )
    
    def SetGridCells(self, width, height, gridCells):
        '''
        Initialization of map by list of cells.
        '''
        self.width = width
        self.height = height
        self.cells = gridCells

    def InBounds(self, i, j):
        '''
        Check if the cell is on a grid.
        '''
        return (0 <= j < self.width) and (0 <= i < self.height)
    
    def Traversable(self, i, j):
        '''
        Check if the cell is not an obstacle.
        '''
        return not self.cells[i][j]

    def GetNeighbours(self, i, j):
        '''
        Returns a list of neighbouring cells as (i,j) tuples. 
        Fucntions should returns such neighbours, that allows both cardinal and diagonal moves, 
        but dissalows cutting corners and squezzing. 

        Parameters:
            i, j (int, int): Position on the grid map, for which neighbours are needed

        Returns:
            neighbours(list[(int, int)]): List of neighbours grid map (i, j) coordinates.   
        '''
        neighbors = []
        for ki in range(i - 1, i + 2):
            for kj in range(j - 1, j + 2):
                if ((i, j) != (ki, kj) and self.InBounds(ki, kj) and self.Traversable(ki, kj)):
                    if (abs(ki + kj - i - j) == 1):
                        neighbors.append((ki, kj))
                    elif (True):
                        if (self.Traversable(i, kj) and self.Traversable(ki, j)):
                            neighbors.append((ki, kj))
                    #neighbors.append((ki, kj))
        return neighbors

    def CalculateCost(self, i1, j1, i2, j2):
        '''                    
        Computes cost of simple moves between cells on grid, 
        or euclidean distance, if there isnt a simple move between cells

        Args:
            i1, j1 (int, int): Positions of first cell on the grid map
            i2, j2 (int, int): Positions of first cell on the grid map
   

        Returns:
            [float]: cost of simple moves or euclidean distance.   
        '''

        if(abs(i1 - i2) + abs(j1 - j2) == 2):
            return 1.41421356237
        elif (abs(i1 - i2) + abs(j1 - j2) == 1):
            return 1.0
        else:
            return math.sqrt(abs(i1 - i2) ** 2 + abs(j1 - j2) ** 2)


In [3]:
class Node:
    '''
    Class represents a grid search node

    - i, j: coordinates of corresponding grid element
    - g: g-value of the node
    - h: h-value of the node
    - F: f-value of the node
    - parent: pointer to the parent-node 
    '''

    def __init__(self, coord, g = math.inf, h = math.inf, F = None, parent = None):
        self.i = coord[0]
        self.j = coord[1]
        self.g = float(g)
        self.h = float(h)
        if F is None:
            self.F = self.g + h
        else:
            self.F = F        
        self.parent = parent
    
    def __eq__(self, other):
        return (self.i == other.i) and (self.j == other.j)
    
    def __lt__(self, other):
        return (self.F < other.F) or ((self.F == other.F) and (self.g < other.g))


In [4]:
def EuclidDistance(node1, node2):
    '''
    Returns a euclidean distance between cells corresponding to nodes
    '''
    return math.sqrt((node1.i - node2.i) ** 2 + (node1.j - node2.j) ** 2)


In [5]:
def MakePath(goal):
    '''
    Creates a path by tracing parent pointers from the goal node to the start node
    It also returns path's length.
    '''

    length = goal.g
    current = goal
    path = []
    while current.parent:
        path.append(current)
        current = current.parent
    path.append(current)
    return path[::-1], length

## Implementing OPEN and CLOSED

In [6]:
class Open:

    def __init__(self):
        self.elements = dict()
        self.sortF = dict()


    def __iter__(self):
        return iter(self.elements)


    def __len__(self):
        return len(self.elements)

    def IsEmpty(self):
        '''
        IsEmpty should inform whether the OPEN is exhausted or not in the former case the search main loop should be interrupted
        '''
        return self.__len__() == 0

    def AddNode(self, item : Node, *args):
        '''
        AddNode is the method that puts (e.g. inserts or updates) the node to OPEN
        When implementing it do not forget to handle all possible cases:
         - node already in OPEN but the new g-value is better;
         - node already in OPEN but the new g-value is worse;
         - node is not in OPEN yet.
        '''
        el = self.elements.get((item.i, item.j))
        if el:
            if el.g > item.g:
                mn = el.F
                self.elements[(item.i, item.j)] = Node((item.i, item.j), item.g, item.h, item.F, item.parent)
                self.sortF[mn].remove((item.i, item.j))
                if (len(self.sortF[mn]) == 0):
                    self.sortF.pop(mn)

                if self.sortF.get(item.F):
                    self.sortF.update({item.F: self.sortF[item.F] + [(item.i, item.j)]})
                else:
                    self.sortF.update({item.F: [(item.i, item.j)]})
            return
        self.elements.update({(item.i, item.j): item})

        if self.sortF.get(item.F):
            self.sortF.update({item.F: self.sortF[item.F] + [(item.i, item.j)]})
        else:
            self.sortF.update({item.F: [(item.i, item.j)]})
        return

    def GetBestNode(self, *args):
        '''
        GetBestNode is the method that 
         i) finds the best node, i.e. the one with the lowest f-value (f=g+h) (for Dijkstra h=0),
         ii) removes it from OPEN and 
         iii) returns it
        '''
        mn = min(self.sortF)
        k = self.sortF[mn][0]
        if (len(self.sortF[mn])) == 1:
            self.sortF.pop(mn)
        else:
            self.sortF.update({mn: self.sortF[mn][1:]})
        best = self.elements.pop(k)
        return best
    
    def UniteAndUpdate(self, other, newWeight):
        '''
        Unite Open set with other container and update F-values of nodes with new weight:
        
        f(Node) = g(Node) + weignt * h(Node)
        '''

        self.elements.update(other.elements)
        self.sortF.clear()

        #other.elements.clear()
        #other.sortF.clear()

        for i in self.elements:
            self.elements[i].F = self.elements[i].g + newWeight * self.elements[i].h
            if self.sortF.get(self.elements[i].F):
                self.sortF.update({self.elements[i].F: self.sortF[self.elements[i].F] + [i]})
            else:
                self.sortF.update({self.elements[i].F: [i]})

    def WasExpanded(self, item : Node):
        '''
        WasExpanded is the method that checks if a node has been expanded
        '''
        return (item.i, item.j) in self.elements

    def GetNode(self, i, j):
        '''
        Finds node from Closed set by (i, j) coordinates on grid and returns it. Returns None if node was not found.
        '''
        if self.WasExpanded(Node((i, j))):
            return self.elements[(i, j)]
        else:
            return None

In [7]:
class Closed:
    
    def __init__(self):
        self.elements = dict()


    def __iter__(self):
        return iter(self.elements)


    def __len__(self):
        return len(self.elements)


    def AddNode(self, item : Node):
        '''
        AddNode is the method that inserts the node to CLOSED
        '''
        self.elements.update({(item.i, item.j): item})


    def WasExpanded(self, item : Node):
        '''
        WasExpanded is the method that checks if a node has been expanded
        '''
        return (item.i, item.j) in self.elements

    def GetNode(self, i, j):
        '''
        Finds node from Closed set by (i, j) coordinates on grid and returns it. Returns None if node was not found.
        '''
        if self.WasExpanded(Node((i, j))):
            return self.elements[(i, j)]
        else:
            return None


## Weighted A*

In [8]:
def WAStar(startCoord, goalCoord, gridMap, hFunc, weight):
    '''
    Runs weighted A* search algorithm without re-expansion on any domain.

    Parameters:
        startCoord (tuple(int, int)): The start state of search in useful for your implementation form.
        goalCoord (tuple(int, int)): The goal state of search in useful for your implementation form.
        gridMap: An additional domain information (such as grid map).
        hFunc (function): The heuristics function for currently using domain.
        weight (float): The weight of heuristics in F-value computation.

    Returns:
        pathFound (bool): Path was found or not.   
        lastNode (Node): The last node in path. None if path was not found.
    '''

    OPEN = Open()
    CLOSED = Closed()
    start = Node(startCoord, g = 0, h = 0, F = 0)
    OPEN.AddNode(start)
    while not OPEN.IsEmpty():
        curNode = OPEN.GetBestNode()
        CLOSED.AddNode(curNode)
        if ((curNode.i, curNode.j) == goalCoord):
            return True, curNode
        neighbors = gridMap.GetNeighbours(curNode.i, curNode.j)
        for n in neighbors:
            newNode = Node(n)
            newNode.g = curNode.g + gridMap.CalculateCost(curNode.i, curNode.j, newNode.i, newNode.j)
            newNode.h = hFunc(newNode, Node(goalCoord))
            newNode.F = newNode.g + weight * newNode.h
            newNode.parent = curNode
            if CLOSED.WasExpanded(newNode):
                continue
            OPEN.AddNode(newNode)
    return False, None


## Naive Anytime A*

In [9]:
def NaiveATAStar(startCoord, goalCoord, gridMap, hFunc, startWeight = 3.0, stepWeight = 0.5):
    '''
    Repeatedly runs weighted A* search algorithm without re-expansion on any domain, 
    decreasing current weight from startWeight to 1.0 by stepWeight.

    Parameters:
        startCoord (tuple(int, int)): The start state of search in useful for your implementation form.
        goalCoord (tuple(int, int)): The goal state of search in useful for your implementation form.
        gridMap: An additional domain information (such as grid map).
        hFunc (function): The heuristics function for currently using domain.
        startWeigth (float): The initial weight of heuristics in F-value computation. Must be greater or equal to 1.0
        stepWeigth (float): The value by which the weight will be reduced, if it is provided by the algorithm 

    Returns:
        pathFound (bool): Path was found or not.   
        lastNode (Node): The last node in path. None if path was not found.
    '''

    weight = startWeight if (startWeight >= 1) else 1
    result = WAStar(startCoord, goalCoord, gridMap, hFunc, weight)
    yield result

    while True:
        result = WAStar(startCoord, goalCoord, gridMap, hFunc, weight)
        yield result
        if weight <= 1:
            break
        weight = (weight - stepWeight) if (weight - stepWeight) >= 1 else 1


## Anytime Repairing A*

In [10]:
def ComputeActualWeight(OPEN, INCONS, weight, goalNode):
    minOpen = math.inf
    minIncons = math.inf
    OP = Open()
    OP.UniteAndUpdate(OPEN, 1)
    EN = Open()
    EN.UniteAndUpdate(INCONS, 1)
    if not OPEN.IsEmpty():
        minOpen = min(OP.sortF)#min(v.g + v.h for v in OPEN)
    if not INCONS.IsEmpty():
        minIncons = min(EN.sortF)#min(v.g + v.h for v in INCONS)
    minG = min(minOpen, minIncons)

    return min(weight, goalNode.g / minG)

def ImprovePath(goalNode, gridMap, hFunc, OPEN:Open, INCONS:Open, CLOSED:Closed, VISITED:Open, weight):
#TODO
    '''
    Runs a modified version of weighted A* search algorithm without re-expansion on any domain.

    Parameters:
        goalNode: The goal state.
        gridMap: An additional domain information (such as grid map).
        hFunc (function): The heuristics function for currently using domain.
        weight (float): The weight of heuristics in F-value computation.

    Returns:
        pathFound (bool): Path was found or not.
        lastNode (Node): The last node in path. None if path was not found.
    '''
    #if (weight <= 1):
    #    print('goalNode.g = ', goalNode.g)
    #print("Improve path", end=' ')
    while (goalNode.g > min(OPEN.sortF)):
        curNode = OPEN.GetBestNode()
        #if (weight <= 1):
        #    print('{:5.3f} {:5.3f} {:5.3f} {:5.3f}'.format(curNode.i, curNode.j, curNode.g, curNode.F))
        CLOSED.AddNode(curNode)
        if ((curNode.i, curNode.j) == (goalNode.i, goalNode.j)):
            #print(goalNode.g)
            #print('complete ', curNode.g)
            return True, curNode, OPEN, INCONS, CLOSED, VISITED
        neighbors = gridMap.GetNeighbours(curNode.i, curNode.j)
        for n in neighbors:
            newNode = VISITED.GetNode(n[0], n[1])
            if not newNode:
                newNode = Node(n)

            if (newNode.g > curNode.g + gridMap.CalculateCost(curNode.i, curNode.j, newNode.i, newNode.j)):
                newNode.g = curNode.g + gridMap.CalculateCost(curNode.i, curNode.j, newNode.i, newNode.j)
                newNode.h = hFunc(newNode, Node((goalNode.i, goalNode.j)))
                newNode.F = newNode.g + weight * newNode.h
                newNode.parent = curNode
                if CLOSED.WasExpanded(newNode):
                    INCONS.AddNode(newNode)
                else:
                    OPEN.AddNode(newNode)
            VISITED.AddNode(newNode)
    #print(goalNode.g)
    #print('complete')
    return True, goalNode, OPEN, INCONS, CLOSED, VISITED


def ARAStar(startCoord, goalCoord, gridMap, hFunc, startWeight = 3.0, stepWeight = 0.5):
    '''
    Runs Anytime Repairing A* search algorithm.

    Parameters:
        startCoord (tuple(int, int)): The start state of search in useful for your implementation form.
        goalCoord (tuple(int, int)): The goal state of search in useful for your implementation form.
        gridMap: An additional domain information (such as grid map).
        hFunc (function): The heuristics function for currently using domain.
        startWeigth (float): The initial weight of heuristics in F-value computation. Must be greater or equal to 1.0
        stepWeigth (float): The value by which the weight will be reduced, if it is provided by the algorithm  (not used)

    Returns:
        pathFound (bool): Path was found or not.
        lastNode (Node): The last node in path. None if path was not found.
    '''
    OPEN = Open()
    INCONS = Open()
    CLOSED = Closed()
    VISITED = Open()
    start = Node(startCoord, g = 0)
    finish = Node(goalCoord)
    start.h = hFunc(start, Node(goalCoord))
    start.F = start.g + startWeight * start.h
    finish.h = 0
    finish.F = finish.g + startWeight * start.h
    OPEN.AddNode(start)
    weight = startWeight
    #print('Before: ', OPEN.elements.keys(), ' ', INCONS.elements.keys(), ' ', CLOSED.elements.keys())
    flag, lnd, OPEN, INCONS, CLOSED, VISITED = ImprovePath(finish, gridMap, hFunc, OPEN, INCONS, CLOSED, VISITED, weight)
    #print('finish.g = ', lnd.F)
    finish = Node((lnd.i, lnd.j), h=lnd.h, g=lnd.g, F=lnd.F, parent=lnd.parent)
    #print('After: ', OPEN.elements.keys(), ' ', INCONS.elements.keys(), ' ', CLOSED.elements.keys())
    check = ComputeActualWeight(OPEN, INCONS, weight, finish)
    yield (flag, finish)

    while check > 1:
        weight -= stepWeight
        weight = min(check, weight)
        weight = max(1.0, weight)
        OPEN.UniteAndUpdate(INCONS, weight)
        INCONS.elements.clear()
        INCONS.sortF.clear()
        CLOSED.elements.clear()
        #print('Before: ', OPEN.elements.keys(), ' ', INCONS.elements.keys(), ' ', CLOSED.elements.keys())
        flag, lnd, OPEN, INCONS, CLOSED, VISITED = ImprovePath(finish, gridMap, hFunc, OPEN, INCONS, CLOSED, VISITED, weight)
        finish = Node((lnd.i, lnd.j), h=lnd.h, g=lnd.g, F=lnd.F, parent=lnd.parent)
        #print('After: ', OPEN.elements.keys(), ' ', INCONS.elements.keys(), ' ', CLOSED.elements.keys())
        #print('finish.g = ', finish.g)
        #print(weight)
        check = ComputeActualWeight(OPEN, INCONS, weight, finish)
        #print('goalNode.g = ', finish.g)
        #print('check = ', check)
        yield (flag, finish)

# Experiment

In [11]:
def SimpleTest(SearchGenerator, task, heuristicFunction, startWeigth, stepWeigth, maxTime):
    '''
    SimpleTest runs SearchGenerator on one task (use a number from 0 to 25 to 
    choose a certain debug task on simple map or None to choose a random task 
    from this pool) with *args as optional arguments and displays:
     - 'Path found!' and some statistics -- path was found
     - 'Path not found!' -- path was not found
     - 'Execution error' -- an error occurred while executing the SearchFunction In first two cases function also draws visualisation of the task

    Parameters:
        SearchGenerator (generator):
        task (int):
        heuristicFunction (function): The heuristics function for currently using domain.
        startWeigth (float): The initial weight of heuristics in F-value computation. Must be greater or equal to 1.0
        stepWeigth (float): The value by which the weight will be reduced, if it is provided by the algorithm 
        maxTime (float): The amount of time given for the pathfinding procedure. Set in seconds.  

    Returns:
        pathFound (bool): Path was found or not.   
        length (Node): The length of the last found path.
        w (int): The sub-optimality value of the last found path.. 
    '''
    
    height = 15
    width = 30
    mapstr = '''
. . . # # . . . . . . . . # # . . . # . . # # . . . . . . .  
. . . # # # # # . . # . . # # . . . . . . # # . . . . . . . 
. . . . . . . # . . # . . # # . . . # . . # # . . . . . . . 
. . . # # . . # . . # . . # # . . . # . . # # . . . . . . . 
. . . # # . . # . . # . . # # . . . # . . # # . . . . . . . 
. . . # # . . # . . # . . # # . . . # . . # # # # # . . . . 
. . . # # . . # . . # . . # # . . . # . . # # # # # . . . . 
. . . . . . . # . . # . . # # . . . # . . # . . . . . . . . 
. . . # # . . # . . # . . # # . . . # . . # . . . . . . . . 
. . . # # . . # . . # . . # # . . . # . . # . . . . . . . . 
. . . # # . . . . . # . . . . . . . # . . . . . . . . . . . 
. . . # # # # # # # # # # # # # . # # . # # # # # # # . # # 
. . . # # . . . . . . . . # # . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . # # . . . . . . . . # # . . . . . . . . . . . . . . .
'''

    taskMap = Map()
    taskMap.ReadFromString(mapstr, width, height)
    starts = [(0, 1), (6, 2), (5, 6), (13, 7), (4, 23)]
    goals = [(1, 28), (2, 29), (3, 20), (3, 20), (0, 0)]
    lengths = [47.8994949, 40.8994949, 38.2426407, 21.8284271, 47.4852814]

    if (task is None) or not (0 <= task < 5):
        task = randint(0, 4)

    start = Node(starts[task])
    goal = Node(goals[task])
    length = lengths[task]

    trueResult = WAStar((start.i, start.j), (goal.i, goal.j), taskMap, heuristicFunction, 1) 
    path = MakePath(trueResult[1])
    print("A* Path found! Length: {:.7f}. Correct: {:s}".format(path[1], str(abs(path[1] - length) < 1e-6)))

    start_time = time.time()
    lastResult = False, None

    iter_count = 0
    timer = 0.0
    finale_timer = 0.0
    w = 0.0
    for result in SearchGenerator((start.i, start.j), (goal.i, goal.j), taskMap, heuristicFunction, startWeigth, stepWeigth):
        iter_count += 1
        timer = time.time() - start_time
        
        if timer < maxTime:
            lastResult = result
            finale_timer = timer
            if lastResult[0]:
                wLength = lastResult[1].g
                w = wLength / length
                #print(iter_count, "iter. Real sub-optimality value: {:.5f}".format(w))
                print("{:2d} iter. Real sub-optimality value: {:.5f}".format(iter_count, w))
                if w - 1 < EPS:
                    break
            else:
                #print(iter_count, "iter. Weighted path not found!")
                print("{} iter. Weighted path not found!".format(iter_count))
        else:
            break
    
    if lastResult[0]:
        wLength = lastResult[1].g
        w = wLength/length
        print("Weighted path found! Finale sub-optimality value: {:.7f}. Iterations: {:d}. Time: {:.7}".format(w, iter_count, finale_timer))
        return True, wLength, w
    else:
        print("Weighted path not found!")
        return False, math.inf, math.inf


In [12]:
for i in range(5):
    SimpleTest(NaiveATAStar, i, EuclidDistance, 5, 0.05, 0.5)

A* Path found! Length: 47.8994949. Correct: True
 1 iter. Real sub-optimality value: 1.60692
 2 iter. Real sub-optimality value: 1.60692
 3 iter. Real sub-optimality value: 1.60692
 4 iter. Real sub-optimality value: 1.60692
 5 iter. Real sub-optimality value: 1.60692
 6 iter. Real sub-optimality value: 1.60692
 7 iter. Real sub-optimality value: 1.60692
 8 iter. Real sub-optimality value: 1.60692
 9 iter. Real sub-optimality value: 1.60692
10 iter. Real sub-optimality value: 1.60692
11 iter. Real sub-optimality value: 1.60692
12 iter. Real sub-optimality value: 1.60692
13 iter. Real sub-optimality value: 1.60692
14 iter. Real sub-optimality value: 1.60692
15 iter. Real sub-optimality value: 1.60692
16 iter. Real sub-optimality value: 1.35133
17 iter. Real sub-optimality value: 1.35133
18 iter. Real sub-optimality value: 1.35133
19 iter. Real sub-optimality value: 1.35133
20 iter. Real sub-optimality value: 1.35133
21 iter. Real sub-optimality value: 1.35133
22 iter. Real sub-optimalit

In [13]:
for i in range(5):
    SimpleTest(ARAStar, i, EuclidDistance, 5, 0.05, 0.1)

A* Path found! Length: 47.8994949. Correct: True
 1 iter. Real sub-optimality value: 1.60692
 2 iter. Real sub-optimality value: 1.60692
 3 iter. Real sub-optimality value: 1.60692
 4 iter. Real sub-optimality value: 1.60692
 5 iter. Real sub-optimality value: 1.60692
 6 iter. Real sub-optimality value: 1.08351
 7 iter. Real sub-optimality value: 1.06621
 8 iter. Real sub-optimality value: 1.00000
Weighted path found! Finale sub-optimality value: 1.0000000. Iterations: 8. Time: 0.02401614
A* Path found! Length: 40.8994949. Correct: True
 1 iter. Real sub-optimality value: 1.71079
 2 iter. Real sub-optimality value: 1.71079
 3 iter. Real sub-optimality value: 1.71079
 4 iter. Real sub-optimality value: 1.71079
 5 iter. Real sub-optimality value: 1.09780
 6 iter. Real sub-optimality value: 1.07755
 7 iter. Real sub-optimality value: 1.00000
Weighted path found! Finale sub-optimality value: 1.0000000. Iterations: 7. Time: 0.0159936
A* Path found! Length: 38.2426407. Correct: True
 1 iter.